In [8]:
pip install scikit-learn

     ---------------------------------------- 11.0/11.0 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 301.8/301.8 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 46.2/46.2 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import cv2
import json
import numpy as np
import pandas as pd
from urllib.parse import urlparse
import tensorflow as tf
from keras import ops
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split

In [2]:
def parse_url(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    return f"{domain}.jpg"

def load_data():
    # Implement loading and preprocessing your dataset 
   
    data = pd.read_csv("../dataset/annotations/annotations.csv")
    data = data[['id','image','score']]
        
    images = []
    for i in range(0,len(data)):
        filepath = data['image'][i]
        image = cv2.imread('../dataset/images/{0}'.format(filepath),cv2.IMREAD_COLOR)

        if image is not None:
            height, width,color = image.shape
            new_dimensions = (224, 224)
            resized_image = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_AREA)
            images.append(resized_image)
        #images.append(cv2.imread('../dataset/images/{0}'.format(filepath),0))
    
    return np.array(images), np.array(data['score'])

In [3]:
images, ratings = load_data()
images = images #/ 255.0  # normalize images

print(images.shape)

X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

print(y_train)


(107, 224, 224, 3)
[82. 29. 57. 57. 79. 54. 79. 57. 57. 79. 61. 57. 43. 64. 82. 57. 46. 61.
 79. 57. 57. 68. 75. 39. 57. 54. 82. 71. 79. 57. 75. 57. 71. 71. 79. 57.
 79. 57. 79. 54. 75. 61. 50. 75. 57. 79. 79. 57. 57. 36. 82. 79. 57. 57.
 64. 79. 79. 57. 79. 79. 79. 71. 79. 57. 71. 64. 57. 57. 57. 61. 57. 57.
 57. 82. 79. 79. 79. 57. 57. 79. 79. 75. 57. 57. 79.]


In [4]:
base_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224,224,3)
    )

# Add custom top layers for regression
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # Output layer for regression

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 3587.5596 - mae: 58.1205 - val_loss: 636.9095 - val_mae: 22.2270
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 698ms/step - loss: 917.5983 - mae: 25.0705 - val_loss: 904.8133 - val_mae: 24.9562
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 654ms/step - loss: 1024.7097 - mae: 27.1475 - val_loss: 1242.6514 - val_mae: 29.3832
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 647ms/step - loss: 1058.9689 - mae: 27.0747 - val_loss: 476.9845 - val_mae: 18.1266
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 668ms/step - loss: 547.9591 - mae: 18.6434 - val_loss: 210.9815 - val_mae: 12.3923
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 678ms/step - loss: 376.9272 - mae: 15.2365 - val_loss: 216.6252 - val_mae: 12.0826
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 689ms/step - loss: 393.3210 - mae: 16.1507 - val_loss: 192.9624 - val_mae: 11.9847
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 713ms/step - loss: 344.5719 - mae: 15.2875 - val_loss: 189.5412 - val_mae: 12.4542
Epoch 9/20
3/3 ━━━━━━━━

In [5]:
for layer in base_model.layers[-10:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mean_squared_error', metrics=['mae'])
history_fine = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 260.1925 - mae: 13.1465 - val_loss: 177.3929 - val_mae: 11.2885
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 738ms/step - loss: 247.4779 - mae: 12.8840 - val_loss: 176.5427 - val_mae: 11.2932
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 764ms/step - loss: 231.8581 - mae: 12.6309 - val_loss: 176.3533 - val_mae: 11.3224
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 724ms/step - loss: 257.5654 - mae: 13.3896 - val_loss: 176.2360 - val_mae: 11.4745
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 736ms/step - loss: 205.2008 - mae: 11.2365 - val_loss: 177.4197 - val_mae: 11.6229
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 715ms/step - loss: 188.4834 - mae: 10.7370 - val_loss: 178.5704 - val_mae: 11.7454
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 713ms/step - loss: 209.2229 - mae: 11.5071 - val_loss: 180.6204 - val_mae: 11.9036
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 726ms/step - loss: 161.2035 - mae: 9.5924 - val_loss: 182.8424 - val_mae: 12.0415
Epoch 9/10
3/3 ━━━━━━━━━━━━━

In [23]:

# Define the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(236, 640, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()


d:\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 234, 638, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 117, 319, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 117, 319, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 115, 317, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 57, 158, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 57, 158, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576384)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    73,777,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,796,225 (281.51 MB)

 Trainable params: 73,796,225 (281.51 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 24998.1387 - mae: 123.8770 - val_loss: 4795.6929 - val_mae: 68.4240
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 4535.0342 - mae: 65.9566 - val_loss: 3935.3975 - val_mae: 61.6148
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 2649.5830 - mae: 49.5242 - val_loss: 2846.1514 - val_mae: 51.4805
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 1901.4565 - mae: 37.1088 - val_loss: 2108.6006 - val_mae: 42.9800
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 1590.8177 - mae: 34.6957 - val_loss: 1786.4935 - val_mae: 38.5245
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 1775.0415 - mae: 34.1984 - val_loss: 1892.2186 - val_mae: 40.0493
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 1620.1002 - mae: 33.6127 - val_loss: 2293.5781 - val_mae: 45.2883
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 1230.9158 - mae: 29.4024 - val_loss: 2638.8445 - val_mae: 49.2649
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━

In [29]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

# Predict rating for a new screenshot
def predict_rating(screenshot):
    height, width = screenshot.shape
    new_dimensions = (width // 4, height // 4)
    resized_image = cv2.resize(screenshot, new_dimensions, interpolation=cv2.INTER_AREA)
    screenshot = resized_image / 255.0
    screenshot = np.expand_dims(screenshot, axis=0)  # Add batch dimension
    predicted_rating = model.predict(screenshot)
    return predicted_rating[0][0]

# Example usage
new_screenshot = cv2.imread('../dataset/images/{0}'.format('architecturalrecord.png'),0)  # Load your new screenshot
predicted_rating = predict_rating(new_screenshot)
print(f'Predicted Rating: {predicted_rating}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 2012.0925 - mae: 41.7741
Test MAE: 41.774131774902344
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Predicted Rating: 24.574453353881836
